In [124]:
import pandas as pd
import numpy as np
import re #Regex lib
from forex_python.converter import CurrencyRates #currencies lib
from datetime import datetime
import matplotlib

### Read in the dataframes and create a dict {(year, dataframe)}


In [125]:
# Read data frames
data_frames_dict = {}


data_frames_dict[2018] = pd.read_csv('datasets/2018.csv', encoding = "ISO-8859-1").filter(items=['Country', 'LanguageWorkedWith', 'LanguageDesireNextYear']).dropna(thresh=3) 
data_frames_dict[2019] = pd.read_csv('datasets/2019.csv', encoding = "ISO-8859-1").filter (items=['Country',  'LanguageWorkedWith', 'LanguageDesireNextYear']).dropna(thresh=3) 
data_frames_dict[2020] = pd.read_csv('datasets/2020.csv', encoding = "ISO-8859-1").filter (items=['Country',  'LanguageWorkedWith', 'LanguageDesireNextYear']).dropna(thresh=3) 

#preprocess 2021
def check_countryType(country, US_state, UK_country):
    
    if  isinstance(US_state, str):
        return 'United States of America'
    elif isinstance(UK_country, str) or country == 'United Kingdom of Great Britain and Northern Ireland':
        return 'United Kingdom'
    else:
        return country


df_2021 = pd.read_csv('datasets/2021.csv', encoding = "ISO-8859-1").filter (items=['Country', 'US_State', 'UK_Country',  'LanguageHaveWorkedWith', 'LanguageWantToWorkWith'])
df_2021 = df_2021.dropna(subset = ['LanguageHaveWorkedWith', 'LanguageWantToWorkWith'])
df_2021.Country =  df_2021.apply(lambda row : check_countryType(row['Country'],row['US_State'],  
                     row['UK_Country']), axis=1)
df_2021 = df_2021.drop(['US_State', 'UK_Country'], axis = 1).dropna(thresh=3)
data_frames_dict[2021]  = df_2021
                                 
df_2021.head(30)                           


/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Country,LanguageHaveWorkedWith,LanguageWantToWorkWith
0,Slovakia,C++;HTML/CSS;JavaScript;Objective-C;PHP;Swift,Swift
2,Russian Federation,Assembly;C;Python;R;Rust,Julia;Python;Rust
3,Austria,JavaScript;TypeScript,JavaScript;TypeScript
4,United Kingdom,Bash/Shell;HTML/CSS;Python;SQL,Bash/Shell;HTML/CSS;Python;SQL
5,United States of America,C;C#;C++;HTML/CSS;Java;JavaScript;Node.js;Powe...,C#;C++;Go;HTML/CSS;Java;JavaScript;Node.js;Obj...
6,United States of America,HTML/CSS;JavaScript,HTML/CSS;JavaScript;PHP
7,Malaysia,HTML/CSS;JavaScript;PHP;Ruby;SQL;TypeScript,Ruby
8,India,HTML/CSS;JavaScript,HTML/CSS;JavaScript
9,Sweden,C++;Python,Haskell;Python
10,United Kingdom,C#;TypeScript,C#


### obtain developer types 


In [126]:
import codecs
import json
for year, df in data_frames_dict.items():
    df.columns = ['Country',  'LanguageWorkedWith', 'LanguageDesireNextYear']
    df.LanguageWorkedWith = df.LanguageWorkedWith.apply(lambda s: s.split(';'))
    df.LanguageDesireNextYear = df.LanguageDesireNextYear.apply(lambda s: s.split(';'))
    
    #LanguageWorkedWith
    
    df_LanguageWorkedWith = df.filter(items = ['Country',  'LanguageWorkedWith'])
    df_LanguageWorkedWith = df_LanguageWorkedWith.explode('LanguageWorkedWith')
    df_LanguageWorkedWith['LanguageWorkedWithCount'] = pd.Series(np.ones(len(df_LanguageWorkedWith.LanguageWorkedWith), dtype=np.int32), index=df_LanguageWorkedWith.index)  
    df_LanguageWorkedWith = df_LanguageWorkedWith.groupby(by=['Country','LanguageWorkedWith'] , as_index=False).sum()
    
    json_dict = {}
    for k, v in df_LanguageWorkedWith.groupby('Country'):
        inner_dict = {}
        for lang, count in zip(v['LanguageWorkedWith'], v['LanguageWorkedWithCount']):
            inner_dict[lang] = count 
        json_dict[k] = inner_dict
        
        
    with codecs.open("processed_data/languages/{}-LanguageWorkedWith_count_per_country.json".format(year), "w", encoding='utf-8') as outfile:
        json.dump(json_dict, outfile, indent = 4, ensure_ascii=False)
    
    # LanguageDesireNextYear
    df_LanguageDesireNextYear = df.filter(items = ['Country',  'LanguageDesireNextYear'])
    df_LanguageDesireNextYear = df_LanguageDesireNextYear.explode('LanguageDesireNextYear')
    df_LanguageDesireNextYear['LanguageDesireNextYearCount'] = pd.Series(np.ones(len(df_LanguageDesireNextYear.LanguageDesireNextYear),dtype=np.int32), index=df_LanguageDesireNextYear.index)  
    df_LanguageDesireNextYear = df_LanguageDesireNextYear.groupby(by=['Country','LanguageDesireNextYear'] , as_index=False).sum()

    json_dict = {}
    for k, v in df_LanguageDesireNextYear.groupby('Country'):
        inner_dict = {}
        for lang, count in zip(v['LanguageDesireNextYear'], v['LanguageDesireNextYearCount']):
            inner_dict[lang] = count 
        json_dict[k] = inner_dict
        
        
    with codecs.open("processed_data/languages/{}-df_LanguageDesireNextYear_count_per_country.json".format(year), "w", encoding='utf-8') as outfile:
        json.dump(json_dict, outfile, indent = 4, ensure_ascii=False)


        


In [127]:
df_LanguageDesireNextYear


,Country,LanguageDesireNextYear,LanguageDesireNextYearCount
0,Afghanistan,APL,4
1,Afghanistan,Assembly,6
2,Afghanistan,Bash/Shell,7
3,Afghanistan,C,9
4,Afghanistan,C#,12
...,...,...,...
4991,Zimbabwe,Ruby,2
4992,Zimbabwe,Rust,4
4993,Zimbabwe,SQL,10
4994,Zimbabwe,Swift,5


In [128]:
df_LanguageWorkedWith

,Country,LanguageWorkedWith,LanguageWorkedWithCount
0,Afghanistan,APL,7
1,Afghanistan,Assembly,9
2,Afghanistan,Bash/Shell,12
3,Afghanistan,C,11
4,Afghanistan,C#,15
...,...,...,...
4839,Zimbabwe,SQL,10
4840,Zimbabwe,Scala,1
4841,Zimbabwe,Swift,1
4842,Zimbabwe,TypeScript,5
